Load required libraries. int_nmf_model must be in the same directory. If it is not it can be added to pythons path

In [ ]:
import anndata as ad
import scanpy as sc
import numpy as np
import scipy
import sys
sys.path.append("../../CellPie/") # go to parent dir
import cellpie_main as cp
from cellpie_main import intNMF
from matplotlib import pyplot as plt
import scipy.io
import pandas as pd
from scipy import sparse
from sklearn.cluster import KMeans
from sklearn import metrics

import seaborn as sns

In [ ]:
sc.set_figure_params(scanpy=True, dpi=250, dpi_save=150)

In [ ]:
random_state=123

# CellPie

In [ ]:
histo_2 = pd.read_csv('/Users/user/CellPie/Paper_Notebooks/Paper_Notebooks/HER2/H1_labeled_coordinates.tsv',sep='\t')
histo_2.set_index('Row.names')

In [ ]:
# read ST data
adata=sc.read_h5ad('/Users/user/CellPie/Paper_Notebooks/Paper_Notebooks//HER2/her2_h1.h5ad')

In [ ]:
histo_2.index = histo_2['Row.names']
histo_2.index.names = ['']
new_index_values = []

for i in range(len(histo_2)):
    x = histo_2['x'].round().astype(np.int64).iloc[i]
    y = histo_2['y'].round().astype(np.int64).iloc[i]
    new_index_values.append("{0}x{1}".format(x, y))
    
histo_2.index=new_index_values
adata.obs['path_labels'] =histo_2['label']

In [ ]:
adata = adata[adata.obs_names.isin(histo_2.index),:]

In [ ]:
scale_range = np.arange(0.1,4,1)

In [ ]:
from feature_extr import extract_features
extract_features(adata,img_path='/Users/user/CellPie/Paper_Notebooks/Paper_Notebooks/HER2/HE_BT24044_D2.jpg',
                         spot_scale=scale_range)

In [ ]:
from cp_utils import preprocess_data
preprocess_data(adata,min_cells=100)

In [ ]:
adata.obs['path_labels'] =histo_2['label']

In [ ]:
# from cp_utils import model_selection
# k = np.arange(1,30)
# mod_sel = model_selection(adata,k,random_state=random_state,reps=3)

In [ ]:
# res = []

# for i in np.arange(0, 2.01,0.1):
    
#     nmf_model = intNMF(adata,5,epochs = 50, init = 'nndsvd',random_state=random_state,mod1_skew=i)
#     nmf_model.fit(adata)
#     kmeans_cellpie = KMeans(n_clusters=6, init='k-means++',max_iter=500,random_state=random_state).fit(nmf_model.theta[:,:])
#     adata.obs['CellPie']=kmeans_cellpie.labels_.astype(str)

    
#     mut_info=metrics.fowlkes_mallows_score(adata.obs['CellPie'],adata.obs['path_labels'])
#     adj_rand=metrics.adjusted_rand_score(adata.obs['CellPie'],adata.obs['path_labels'])
#     adj_mut_info=metrics.adjusted_mutual_info_score(adata.obs['CellPie'],adata.obs['path_labels'])

    
#     res.append((i,mut_info,adj_rand,adj_mut_info)) 

In [ ]:
# score = pd.DataFrame(res)  
# plt.plot(score[0],score[1],color="green",label='Fowlkes Mallows Score')
# plt.plot(score[0],score[2],color="red",label='Adjusted Rand Score')
# plt.plot(score[0],score[3],color="blue",label='Adjusted Mutual Info Score')
# plt.xlabel("Weight")
# plt.ylabel("Score")
# plt.legend(prop={'size': 9})

In [ ]:
# score[2].idxmax()

In [ ]:
k=5

In [ ]:
# Run CellPie with gene expression modality only
nmf_model_0 = intNMF(adata,k,epochs = 50,init = 'nndsvd',random_state=random_state,mod1_skew=2.0)
nmf_model_0.fit(adata)

In [ ]:
nmf_model = intNMF(adata,k,epochs = 50,init = 'nndsvd',random_state=random_state,mod1_skew=1.9)
nmf_model.fit(adata)

In [ ]:
kmeans_cellpie = KMeans(n_clusters=6, init='k-means++',max_iter=500,random_state=random_state).fit(nmf_model.theta[:,:])
adata.obs['CellPie']=kmeans_cellpie.labels_.astype(str)

In [ ]:
adj_rand=metrics.adjusted_rand_score(adata.obs['CellPie'],adata.obs['path_labels'])
adj_rand

In [ ]:
import matplotlib as mpl
sel_clust = ['Factor_'+str(i+1) for i in range(k)]
with mpl.rc_context({'figure.figsize': (5, 6), 'axes.facecolor': 'black'}):
    sc.pl.spatial(adata,
    cmap='magma',
    color=sel_clust,
    ncols=3,
    size=4, img_key='hires', 
    alpha_img=0.4)

In [ ]:
mefisto = pd.read_csv('/Users/user/CellPie/Paper_Notebooks/Paper_Data/HER2/Benchmark/factors_mefisto_her2_5.csv',index_col=0)
mefisto = mefisto.iloc[mefisto.index.isin(histo_2.index),:]
nsf = pd.read_csv('/Users/user/CellPie/Paper_Notebooks/Paper_Data/HER2/Benchmark/factors_nsf_her2_poi_5.csv',index_col=0)
nsf = nsf.iloc[nsf.index.isin(histo_2.index),:]
nsfh_s = pd.read_csv('/Users/user/CellPie/Paper_Notebooks/Paper_Data/HER2/Benchmark//spatialfactors_nsfh_her2_poi_5.csv',index_col=0)

nsfh_ns = pd.read_csv('/Users/user/CellPie/Paper_Notebooks/Paper_Data/HER2/Benchmark/nonspatialfactors_nsfh_her2_poi_5.csv',index_col=0)

pnmf = pd.read_csv('/Users/user/CellPie/Paper_Notebooks/Paper_Data/HER2/Benchmark/factors_pnmf_her2_poi_5.csv',index_col=0)
pnmf = pnmf.iloc[pnmf.index.isin(histo_2.index),:]

In [ ]:
nsfh_s = nsfh_s.rename(columns={"0": "2", "1": "3", "2": "4"})

In [ ]:
nsfh = pd.concat([nsfh_ns,nsfh_s], axis=1)
nsfh

In [ ]:
nsfh = nsfh.iloc[nsfh.index.isin(histo_2.index),:]

In [ ]:
from sklearn.decomposition import FactorAnalysis
transformer = FactorAnalysis(n_components=k, random_state=random_state)
X_transformed = transformer.fit_transform(adata.X)

In [ ]:
for i in range(k):
    adata.obs[f"FA_{i+1}"] = X_transformed[:,i]
for i in range(k):
    adata.obs[f"nsfh_{i+1}"] = nsfh.iloc[:,i]
for i in range(k):
    adata.obs[f"nsf_{i+1}"] = nsf.iloc[:,i]
for i in range(k):
    adata.obs[f"mefisto_{i+1}"] = mefisto.iloc[:,i]
for i in range(k):
    adata.obs[f"pnmf_{i+1}"] = pnmf.iloc[:,i]

In [ ]:
kmeans_cellpie = KMeans(n_clusters=6, init='k-means++',max_iter=500,random_state=random_state).fit(nmf_model.theta[:,:])
adata.obs['CellPie']=kmeans_cellpie.labels_.astype(str)

kmeans_nsf = KMeans(n_clusters=6, init='k-means++',max_iter=500,random_state=random_state).fit(adata.obs.loc[:,adata.obs.columns.str.contains('nsf_')])
adata.obs['nsf']=kmeans_nsf.labels_.astype(str)

kmeans_nsfh = KMeans(n_clusters=6, init='k-means++',max_iter=500,random_state=random_state).fit(adata.obs.loc[:,adata.obs.columns.str.contains('nsfh_')])
adata.obs['nsfh']=kmeans_nsfh.labels_.astype(str)

kmeans_fa = KMeans(n_clusters=6, init='k-means++',max_iter=500,random_state=random_state).fit(adata.obs.loc[:,adata.obs.columns.str.contains('FA_')])
adata.obs['fa']=kmeans_fa.labels_.astype(str)

kmeans_pnmf = KMeans(n_clusters=6, init='k-means++',max_iter=500,random_state=random_state).fit(adata.obs.loc[:,adata.obs.columns.str.contains('pnmf_')])
adata.obs['pnmf']=kmeans_pnmf.labels_.astype(str)

kmeans_mefisto= KMeans(n_clusters=6, init='k-means++',max_iter=500,random_state=random_state).fit(adata.obs.loc[:,adata.obs.columns.str.contains('mefisto_')])
adata.obs['mefisto']=kmeans_mefisto.labels_.astype(str)

kmeans_nmf = KMeans(n_clusters=6, init='k-means++',max_iter=500,random_state=random_state).fit(nmf_model_0.theta[:,:])
adata.obs['CellPie_expression']=kmeans_nmf.labels_.astype(str)


In [ ]:
sc.pl.spatial(adata,color=['path_labels','CellPie','mefisto','pnmf','nsf','nsfh','fa','CellPie_expression'],size=4,
             ncols=4,
             )

In [ ]:
from sklearn import metrics
l_t = ['CellPie','mefisto','pnmf','nsf','nsfh','fa','CellPie_expression']
res = []
for l_t in l_t:
    mut_info=metrics.fowlkes_mallows_score(adata.obs[l_t],adata.obs['path_labels'])
    adj_rand=metrics.adjusted_rand_score(adata.obs[l_t],adata.obs['path_labels'])
    adj_mut_info=metrics.adjusted_mutual_info_score(adata.obs[l_t],adata.obs['path_labels'])
    res.append((l_t,mut_info,adj_rand,adj_mut_info))
res_df = pd.DataFrame(res, columns = ['Method','Mutual Info','Adjusted Rand Index','Adjusted Mutual Info'])

In [ ]:
res_df

In [ ]:
import seaborn as sns
plot=sns.barplot(data=res_df,x = res_df['Method'],y='Adjusted Rand Index',hue='Adjusted Rand Index',width=0.6, legend=False)
plot.set_xticklabels(plot.get_xticklabels(), fontsize=10)
plt.title('ARI-kmeans Clustering')
plt.grid(False)

### extract gene loading matrix

In [ ]:
l = cp.get_genes_topic(adata,nmf_model.phi_expr)

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(
    pd.crosstab(
       adata.obs['path_labels'].values, adata.obs['CellPie'].values,
        normalize ='index'
    ), 
    cmap=sns.color_palette("crest", as_cmap=True),
    annot = True
)
plt.grid(None)
plt.show()

In [ ]:
tls = pd.read_csv('/Users/user/CellPie/Paper_Notebooks/Paper_Data/HER2/Benchmark/tls_score_H1.csv',index_col=0)

In [ ]:
adata.obs['tls'] = tls['0']

In [ ]:
with mpl.rc_context({'figure.figsize': (5, 6), 'axes.facecolor': 'black'}):
    sc.pl.spatial(adata,color='tls',size=4,alpha_img=0.2,save = 'tls.png')

In [ ]:
factors=adata.obs.iloc[:,3:(3+k)]
mi_scores = np.zeros((1,k))
for i in range(1):
    for j in range(k):
        mi_scores[i,j] = np.corrcoef(adata.obs['tls'],factors.iloc[:, j])[1][0]      
mi_scores_nsf = np.zeros((1,k))
for i in range(1):
    for j in range(k):
        mi_scores_nsf[i,j] = np.corrcoef(adata.obs['tls'],nsf.iloc[:, j])[1][0]      
mi_scores_nsfh = np.zeros((1,k))
for i in range(1):
    for j in range(k):
        mi_scores_nsfh[i,j] = np.corrcoef(adata.obs['tls'],nsfh.iloc[:, j])[1][0]    
mi_scores_mef = np.zeros((1,k))
for i in range(1):
    for j in range(k):
        mi_scores_mef[i,j] = np.corrcoef(adata.obs['tls'],mefisto.iloc[:, j])[1][0]
mi_scores_fa = np.zeros((1,k))
for i in range(1):
    for j in range(k):
        mi_scores_fa[i,j] = np.corrcoef(adata.obs['tls'],X_transformed[:, j])[1][0]
mi_scores_pnmf = np.zeros((1,k))
for i in range(1):
    for j in range(k):
        mi_scores_pnmf[i,j] = np.corrcoef(adata.obs['tls'],pnmf.iloc[:, j])[1][0]

In [ ]:
plt.figure(figsize=(33, 1))
plt.rcParams["axes.grid"] = False
sns.heatmap(mi_scores, annot=True, cmap='viridis',
            xticklabels=factors.columns,annot_kws={"fontsize":35})
sns.set(font_scale=2.5)
plt.title('Pearson Correlation Between TLS score and CellPie Factors')
plt.xlabel('Factors')
plt.ylabel('TLS')
plt.show()

In [ ]:
plt.figure(figsize=(33, 1))
plt.rcParams["axes.grid"] = False
sns.heatmap(mi_scores_nsf, annot=True, cmap='viridis',
            xticklabels=factors.columns,annot_kws={"fontsize":35})
sns.set(font_scale=2.5)
plt.title('Pearson Correlation Between TLS score and NSF Factors')
plt.xlabel('Factors')
plt.ylabel('TLS')
plt.show()

In [ ]:
plt.figure(figsize=(33, 1))
plt.rcParams["axes.grid"] = False
sns.heatmap(mi_scores_nsfh, annot=True, cmap='viridis',
            xticklabels=factors.columns,annot_kws={"fontsize":35})
sns.set(font_scale=2.5)
plt.title('Pearson Correlation Between TLS score and NSFH Factors')
plt.xlabel('Factors')
plt.ylabel('TLS')
plt.show()

In [ ]:
plt.figure(figsize=(33, 1))
plt.rcParams["axes.grid"] = False
sns.heatmap(mi_scores_fa, annot=True, cmap='viridis',
            xticklabels=factors.columns,annot_kws={"fontsize":35})
sns.set(font_scale=2.5)
plt.title('Pearson Correlation Between TLS score and FA Factors')
plt.xlabel('Factors')
plt.ylabel('TLS')
plt.show()

In [ ]:
plt.figure(figsize=(33, 1))
plt.rcParams["axes.grid"] = False
sns.heatmap(mi_scores_pnmf, annot=True, cmap='viridis',
            xticklabels=factors.columns,annot_kws={"fontsize":35})
sns.set(font_scale=2.5)
plt.title('Pearson Correlation Between TLS score and PNMF Factors')
plt.xlabel('Factors')
plt.ylabel('TLS')
plt.show()

In [ ]:
plt.figure(figsize=(33, 1))
plt.rcParams["axes.grid"] = False
sns.heatmap(mi_scores_mef, annot=True, cmap='viridis',
            xticklabels=factors.columns,annot_kws={"fontsize":35})
sns.set(font_scale=2.5)
plt.title('Pearson Correlation Between TLS score and MEFISTO Factors')
plt.xlabel('Factors')
plt.ylabel('TLS')
plt.show()

In [ ]:
import gseapy as gp
def gene_er(topic):
    enr = gp.enrichr(gene_list=topic,
                gene_sets=['GO_Biological_Process_2023'],cutoff = 0.5,
                organism='human', 
                outdir=None,
                )
    return enr

In [ ]:
l

In [ ]:
l=cp.get_genes_topic(adata,nmf_model.phi_expr)
enr0 = gene_er(l.T[0].sort_values(ascending=False).index[0:150].to_list())
enr1 = gene_er(l.T[1].sort_values(ascending=False).index[0:150].to_list())
enr2 = gene_er(l.T[2].sort_values(ascending=False).index[0:150].to_list())
enr3 = gene_er(l.T[3].sort_values(ascending=False).index[0:150].to_list())
enr4 = gene_er(l.T[4].sort_values(ascending=False).index[0:150].to_list())

In [ ]:
gp.barplot(enr2.res2d,title='GO_Biological_Process_2023 Factor 3',color=['darkred'])
plt.grid(None)

In [ ]:
gp.barplot(enr1.res2d,title='GO_Biological_Process_2023 Factor 2',color=['darkred'])
plt.grid(None)

In [ ]:
from re import sub
import matplotlib as mpl
sel_clust = ['nsf_'+str(i+1) for i in range(k)]
with mpl.rc_context({'figure.figsize': (5, 6), 'axes.facecolor': 'black'}):
    sc.pl.spatial(adata,
                cmap='magma',
                color=sel_clust,
                ncols=3,
                size=4, img_key='hires', 
                alpha_img=0
                 )

In [ ]:
from re import sub
import matplotlib as mpl
sel_clust = ['pnmf_'+str(i+1) for i in range(k)]
with mpl.rc_context({'figure.figsize': (5, 6), 'axes.facecolor': 'black'}):
    sc.pl.spatial(adata,
                cmap='magma',
                color=sel_clust,
                ncols=3,
                size=4, img_key='hires', 
                alpha_img=0
                 )

In [ ]:
from re import sub
import matplotlib as mpl
sel_clust = ['FA_'+str(i+1) for i in range(k)]
with mpl.rc_context({'figure.figsize': (5, 6), 'axes.facecolor': 'black'}):
    sc.pl.spatial(adata,
                cmap='magma',
                color=sel_clust,
                ncols=3,
                size=4, img_key='hires', 
                alpha_img=0
                 )

In [ ]:
from re import sub
import matplotlib as mpl
sel_clust = ['nsfh_'+str(i+1) for i in range(k)]
with mpl.rc_context({'figure.figsize': (5, 6), 'axes.facecolor': 'black'}):
    sc.pl.spatial(adata,
                cmap='magma',
                color=sel_clust,
                ncols=3,
                size=4, img_key='hires', 
                alpha_img=0
                 )

In [ ]:
from re import sub
import matplotlib as mpl
sel_clust = ['mefisto_'+str(i+1) for i in range(k)]
with mpl.rc_context({'figure.figsize': (5, 6), 'axes.facecolor': 'black'}):
    sc.pl.spatial(adata,
                cmap='magma',
                color=sel_clust,
                ncols=3,
                size=4, img_key='hires',
                alpha_img=0
                 )